# Sample Workflow for d3rlpy Experiments

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import math
import subprocess
import os
import d3rlpy
plt.style.use('matplotlibrc')

from Python.data_sampler_confounder import *

## Building an MDPDataset

We first read in a large batch of samples from the file. As `d3rlpy` wants it in the form (observations, actions, rewards, terminal flags), we go ahead and do that. Here's a helper function to get a dataset from a list of chunks of your choosing.

In [2]:
def get_dataset(chunks : list, batch_size=30000, 
                path="collected_data/rl_det_small.txt") -> d3rlpy.dataset.MDPDataset :
    random.seed(0)
    samples = DataSampler(path_to_data=path)
    samples.setting("coarse")
    states = []
    actions = []
    rewards = []
    metrics = []
    next_states = []
    for chunk in chunks:
        samples.use_chunk(chunk)
        samples.read_chunk()
        [statesChunk, actionsChunk, rewardsChunk, nextStatesChunk, metricChunk] = samples.get_batch(batch_size)
        states.append(statesChunk)
        actions.append(actionsChunk)
        rewards.append(rewardsChunk)
        next_states.append(nextStatesChunk)
        metrics.append(metricChunk)
    states = torch.cat(states)
    actions = torch.cat(actions)
    rewards = torch.cat(rewards)
    next_states = torch.cat(next_states)
    metrics = np.hstack(metrics)
    
    terminals = np.zeros(len(states))
    terminals[::1111] = 1 #episode length 100, change if necessary
    dataset = d3rlpy.dataset.MDPDataset(np.hstack([states.numpy(), metrics[:,None]]), 
                                        actions.numpy(), 
                                        rewards.numpy(),
                                        terminals)
    return dataset, states.numpy(), actions.numpy(), rewards.numpy(), metrics

We can build the dataset from there, just like this, and split into train and test sets.

In [3]:
dataset, statesOrig, actions, rewards, metrics = get_dataset([i for i in range(1000)], path="collected_data/rl_stochpid.txt")

start
[ 0.00000000e+00  7.95731469e+08 -4.75891077e-02 -3.69999953e-02
  2.00999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.50429671e-01 -4.92727243e-01 -5.31666025e-03 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 1 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.25610892e-01 -3.35999953e-02
 -2.42000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -3.08749986e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 2 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.90489108e-01 -5.87999953e-02
 -1.01000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.76979602e-02 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+0

start
[ 0.00000000e+00  7.95731469e+08  2.74610892e-01  1.40000047e-02
  7.59998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.58596924e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 42 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.29289108e-01 -1.49999953e-02
  1.51999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 43 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -7.03891077e-02 -1.45999953e-02
  2.71999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.45547944e-01 -1.42260606e-01  1.10586690e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e

[ 0.00000000e+00  7.95731469e+08  3.81410892e-01  4.20000469e-03
  2.96999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 83 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.15010892e-01  2.00000469e-03
 -2.16000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.16022107e-01  3.75612115e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 84 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.51610892e-01 -2.79999953e-02
  7.09998658e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -7.22183941e-03  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0

[ 0.00000000e+00  7.95731469e+08  2.81510892e-01 -5.59999531e-03
  1.82999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  2.78617263e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 126 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.38089108e-01 -5.87999953e-02
 -2.28000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.05010402e-01 -4.22664253e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 127 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.07889108e-01  2.08000047e-02
  2.89999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

Read chunk # 169 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.08810892e-01 -2.45999953e-02
 -6.40001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  4.40229373e-02  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 170 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  3.28010892e-01  2.18000047e-02
 -5.00001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -4.97408545e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 171 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.64110892e-01  3.64000047e-02
  2.99999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  5.62531137e-01  4.43218747e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+

...

start
[ 0.00000000e+00  7.95731469e+08 -8.92891077e-02 -3.59999531e-03
 -1.07000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.36854482e-01 -3.88110213e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 797 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.34110892e-01 -3.27999953e-02
 -2.75000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  5.45336489e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 798 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.32010892e-01  4.46000047e-02
  2.64999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.10023985e-01  4.11402961e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

start
[ 0.00000000e+00  7.95731469e+08  4.43210892e-01 -3.45999953e-02
 -1.11000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.98925499e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 840 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.85891077e-02 -3.55999953e-02
 -1.07000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  9.92845363e-02  2.31902228e-02  3.64467207e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 841 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  4.02710892e-01 -1.31999953e-02
 -2.19000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -6.00000000e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

[ 0.00000000e+00  7.95731469e+08 -4.06089108e-01 -5.17999953e-02
 -1.20000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.50229477e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 881 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.44289108e-01  2.46000047e-02
  1.88999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  2.25252275e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 882 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.19810892e-01  2.10000047e-02
  1.23999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.07509890e-02  3.81243688e-01 -5.66420114e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

[ 0.00000000e+00  7.95731469e+08 -5.59891077e-02 -5.01999953e-02
  2.85999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -3.95459471e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 921 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.01289108e-01  3.82000047e-02
 -1.08000134e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -5.36003757e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 922 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.61789108e-01  3.98000047e-02
  1.17999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  1.43758421e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

[ 0.00000000e+00  7.95731469e+08 -1.31889108e-01  4.84000047e-02
  2.36999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -4.34342001e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 962 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -3.24189108e-01  1.02000047e-02
  2.12999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 963 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  2.17310892e-01 -1.63999953e-02
 -5.10001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.05230079e-01  6.00000000e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06 

In [4]:
print("The behavior policy value statistics are:")
dataset.compute_stats()['return']

The behavior policy value statistics are:


{'mean': -128.34366,
 'std': 83.63828,
 'min': -410.7058,
 'max': 0.0,
 'histogram': (array([  4,   2,  11,  11,  22,  18,  25,  33,  39,  22,  22,  41,  73,
          76, 103, 118, 146, 179,  54,   1]),
  array([-410.7058  , -390.17053 , -369.63522 , -349.09995 , -328.56464 ,
         -308.02936 , -287.49408 , -266.95877 , -246.4235  , -225.8882  ,
         -205.3529  , -184.81761 , -164.28232 , -143.74704 , -123.21175 ,
         -102.67645 ,  -82.14116 ,  -61.605873,  -41.07058 ,  -20.53529 ,
            0.      ], dtype=float32))}

In [5]:
# plt.plot(states[:,6])

In [6]:
# plt.plot(actions)

In [7]:
#plt.plot(model.predict(np.array(states)))

In [8]:
from sklearn.model_selection import train_test_split
train_episodes, test_episodes = train_test_split(dataset, test_size=5)

## Setting up an Algorithm

In [9]:
from d3rlpy.algos import CQL
from d3rlpy.models.encoders import VectorEncoderFactory

from d3rlpy.preprocessing import MinMaxActionScaler
action_scaler = MinMaxActionScaler(minimum=-0.6, maximum=0.6)
#cql = CQL(action_scaler=action_scaler)

#actor_encoder = VectorEncoderFactory(hidden_units=[12, 24, 36, 24, 12],
#                                       activation='relu', use_batch_norm=True, dropout_rate=0.2)
#critic_encoder = VectorEncoderFactory(hidden_units=[12, 24, 24, 12],
#                                       activation='relu', use_batch_norm=True, dropout_rate=0.2)

model = CQL(q_func_factory='mean', #qr -> quantile regression q function, but you don't have to use this
            reward_scaler='standard',
            action_scaler=action_scaler,
          actor_learning_rate=1e-5, 
          critic_learning_rate=0.0003, 
            use_gpu=False) #change it to true if you have one
model.build_with_dataset(dataset)

In [10]:
from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from d3rlpy.metrics.scorer import initial_state_value_estimation_scorer

# calculate metrics with test dataset
ave_error_init = average_value_estimation_scorer(model, test_episodes)
print(ave_error_init)

0.036240149109481694


In [11]:
%load_ext tensorboard
%tensorboard --logdir runs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "/home/dasc/anaconda3/envs/jbreeden3.10/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/main.py", line 40, in <module>
    from tensorboard import default
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/default.py", line 38, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 25, in <module>
    from tensorboard import plugin_util
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorboard/plugin_util.py", line 21, in <module>
    from tensorboard._vendor import bleach
  File "/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/tensorb

In [12]:
model.fit(train_episodes,
        eval_episodes=test_episodes,
        n_epochs=20, 
        tensorboard_dir='runs',
        scorers={
            'td_error': td_error_scorer,
            'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer
        })

2022-04-19 22:11.02 [debug    ] RoundIterator is selected.
2022-04-19 22:11.02 [info     ] Directory is created at d3rlpy_logs/CQL_20220419221102
2022-04-19 22:11.02 [debug    ] Fitting action scaler...       action_scaler=min_max
2022-04-19 22:11.02 [debug    ] Fitting reward scaler...       reward_scaler=standard
2022-04-19 22:11.02 [warning  ] Skip building models since they're already built.
2022-04-19 22:11.02 [info     ] Parameters are saved to d3rlpy_logs/CQL_20220419221102/params.json params={'action_scaler': {'type': 'min_max', 'params': {'minimum': array(-0.6), 'maximum': array(0.6)}}, 'actor_encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'actor_learning_rate': 1e-05, 'actor_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'alpha_learning_rate': 0.0001, 'alpha_optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight

Epoch 1/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-19 22:17.55 [info     ] CQL_20220419221102: epoch=1 step=4309 epoch=1 metrics={'time_sample_batch': 0.0002942477005157152, 'time_algorithm_update': 0.0949602771120742, 'temp_loss': 3.3563365883162137, 'temp': 0.8274853446694421, 'alpha_loss': -5.3284055914767325, 'alpha': 1.1596206161311342, 'critic_loss': 14.556474138008909, 'actor_loss': 14.42031031683473, 'time_step': 0.09546220493471576, 'td_error': 42.75731813645616, 'init_value': -40.43320083618164, 'ave_value': -42.613514377727164} step=4309
2022-04-19 22:17.55 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220419221102/model_4309.pt


Epoch 2/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-19 22:24.31 [info     ] CQL_20220419221102: epoch=2 step=8618 epoch=2 metrics={'time_sample_batch': 0.00029686355458027414, 'time_algorithm_update': 0.09109495576392418, 'temp_loss': 1.433966846913134, 'temp': 0.5810934323104249, 'alpha_loss': 0.8743829604048756, 'alpha': 1.2082513440205842, 'critic_loss': 58.028899153248844, 'actor_loss': 59.137008949841054, 'time_step': 0.09161658169747948, 'td_error': 106.87199282273092, 'init_value': -89.94698333740234, 'ave_value': -92.61096563358834} step=8618
2022-04-19 22:24.31 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220419221102/model_8618.pt


...

Epoch 19/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-20 00:33.00 [info     ] CQL_20220419221102: epoch=19 step=81871 epoch=19 metrics={'time_sample_batch': 0.0002866302020147522, 'time_algorithm_update': 0.08872183106786055, 'temp_loss': 0.001319684875674825, 'temp': 0.15800843214416038, 'alpha_loss': 0.03141124635772278, 'alpha': 0.3235924220688245, 'critic_loss': 47.11704555173904, 'actor_loss': 43.250848152211994, 'time_step': 0.08922901107086939, 'td_error': 44.971144851405676, 'init_value': -60.14690399169922, 'ave_value': -49.00401668789419} step=81871
2022-04-20 00:33.00 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220419221102/model_81871.pt


Epoch 20/20:   0%|          | 0/4309 [00:00<?, ?it/s]

2022-04-20 00:39.24 [info     ] CQL_20220419221102: epoch=20 step=86180 epoch=20 metrics={'time_sample_batch': 0.00029989416512404994, 'time_algorithm_update': 0.08839562365836681, 'temp_loss': 0.0002608095024154541, 'temp': 0.15719266224762324, 'alpha_loss': 0.024233677144664956, 'alpha': 0.307824973510628, 'critic_loss': 45.30775872274739, 'actor_loss': 38.40102719209343, 'time_step': 0.08892502378604063, 'td_error': 43.14962546233025, 'init_value': -56.522430419921875, 'ave_value': -44.782660068351404} step=86180
2022-04-20 00:39.24 [info     ] Model parameters are saved to d3rlpy_logs/CQL_20220419221102/model_86180.pt


[(1,
  {'time_sample_batch': 0.0002942477005157152,
   'time_algorithm_update': 0.0949602771120742,
   'temp_loss': 3.3563365883162137,
   'temp': 0.8274853446694421,
   'alpha_loss': -5.3284055914767325,
   'alpha': 1.1596206161311342,
   'critic_loss': 14.556474138008909,
   'actor_loss': 14.42031031683473,
   'time_step': 0.09546220493471576,
   'td_error': 42.75731813645616,
   'init_value': -40.43320083618164,
   'ave_value': -42.613514377727164}),
 (2,
  {'time_sample_batch': 0.00029686355458027414,
   'time_algorithm_update': 0.09109495576392418,
   'temp_loss': 1.433966846913134,
   'temp': 0.5810934323104249,
   'alpha_loss': 0.8743829604048756,
   'alpha': 1.2082513440205842,
   'critic_loss': 58.028899153248844,
   'actor_loss': 59.137008949841054,
   'time_step': 0.09161658169747948,
   'td_error': 106.87199282273092,
   'init_value': -89.94698333740234,
   'ave_value': -92.61096563358834}),
 (3,
  {'time_sample_batch': 0.0003005738434312404,
   'time_algorithm_update': 0.0

## Off-Policy Evaluation

We do get some metrics on a test set of initial state value and average value. However, these estimates (using the critic's Q-function) of model performance are biased. They're useful for validation during training, but not much else. Instead, we fit a Q-function to the data (or a separate dataset, as I've done here) separately and evaluate the model's performance on it.

Feel free to change the chunks and number of steps.

In [13]:
from d3rlpy.ope import FQE
# metrics to evaluate with
from d3rlpy.metrics.scorer import soft_opc_scorer


ope_dataset, states_ope, actions_ope, rewards_ope, metrics_ope = get_dataset([500+i for i in range(50)], path="collected_data/rl_stochpid.txt") #change if you'd prefer different chunks
ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=5)

fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
fqe.fit(ope_train_episodes,
        eval_episodes=ope_test_episodes,
        tensorboard_dir='runs',
        n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
        scorers={
           'init_value': initial_state_value_estimation_scorer,
            'ave_value': average_value_estimation_scorer,
           'soft_opc': soft_opc_scorer(return_threshold=0)
        })

start
[ 0.00000000e+00  7.95731469e+08 -4.19889108e-01 -1.83999953e-02
  2.71999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 501 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.30889108e-01 -2.27999953e-02
 -3.80001342e-03  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.19275575e-02 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 502 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -2.99289108e-01 -3.69999953e-02
  2.18999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
 -2.02595006e-01 -6.00000000e-01  6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

start
[ 0.00000000e+00  7.95731469e+08  4.78108923e-02 -2.91999953e-02
  1.03999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  2.09695008e-02 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 542 out of 4999
start
[ 0.00000000e+00  7.95731469e+08  1.13810892e-01  4.16000047e-02
  2.69999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  6.00000000e-01  2.26782935e-01 -6.00000000e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.77814500e+06  0.00000000e+00
  0.00000000e+00  5.33423489e+00 -6.44487563e-06 -2.76958740e-05
 -7.93993673e-06]
Read chunk # 543 out of 4999
start
[ 0.00000000e+00  7.95731469e+08 -4.69891077e-02  1.12000047e-02
  2.97999866e-02  0.00000000e+00 -5.33423489e+00 -1.57091618e+00
  3.70870777e-01 -2.76326953e-01  5.89576724e-01 -1.48980673e+11
  2.13971128e+07  0.00000000e+00  6.7781450

Epoch 1/50:   0%|          | 0/488 [00:00<?, ?it/s]

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


2022-04-20 00:39.26 [info     ] FQE_20220420003924: epoch=1 step=488 epoch=1 metrics={'time_sample_batch': 0.00012289451771095152, 'time_algorithm_update': 0.0027904974632575862, 'loss': 0.006417373936722574, 'time_step': 0.002968227277036573, 'init_value': -0.5982769727706909, 'ave_value': -0.43931858867675333, 'soft_opc': nan} step=488


...

2022-04-20 00:40.53 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220420003924/model_23912.pt


Epoch 50/50:   0%|          | 0/488 [00:00<?, ?it/s]

2022-04-20 00:40.55 [info     ] FQE_20220420003924: epoch=50 step=24400 epoch=50 metrics={'time_sample_batch': 0.00013844986430934218, 'time_algorithm_update': 0.0034278616553447285, 'loss': 0.04401947254793085, 'time_step': 0.0036319538218076114, 'init_value': -4.013291835784912, 'ave_value': -4.730147983963425, 'soft_opc': nan} step=24400


2022-04-20 00:40.55 [info     ] Model parameters are saved to d3rlpy_logs/FQE_20220420003924/model_24400.pt


[(1,
  {'time_sample_batch': 0.00012289451771095152,
   'time_algorithm_update': 0.0027904974632575862,
   'loss': 0.006417373936722574,
   'time_step': 0.002968227277036573,
   'init_value': -0.5982769727706909,
   'ave_value': -0.43931858867675333,
   'soft_opc': nan}),
 (2,
  {'time_sample_batch': 0.00012487124224178127,
   'time_algorithm_update': 0.0029631032318365377,
   'loss': 0.005279628622925795,
   'time_step': 0.0031398119496517493,
   'init_value': -0.7484461069107056,
   'ave_value': -0.5154725129056621,
   'soft_opc': nan}),
 (3,
  {'time_sample_batch': 0.00012738050007429278,
   'time_algorithm_update': 0.003082964263978552,
   'loss': 0.004364420129530147,
   'time_step': 0.0032620928326591117,
   'init_value': -0.8721564412117004,
   'ave_value': -0.5933400834492735,
   'soft_opc': nan}),
 (4,
  {'time_sample_batch': 0.00013102371184552303,
   'time_algorithm_update': 0.0031307862430322367,
   'loss': 0.003956772288504118,
   'time_step': 0.0033208200188933825,
   'in

In [14]:
# from d3rlpy.ope import FQE
# # metrics to evaluate with
# from d3rlpy.metrics.scorer import soft_opc_scorer


# ope_dataset, states_ope, actions_ope, rewards_ope, metrics_ope = get_dataset([2,4,6,8], path="collected_data/rl_stoch_small.txt") #change if you'd prefer different chunks
# ope_train_episodes, ope_test_episodes = train_test_split(ope_dataset, test_size=0.2)

# fqe = FQE(algo=model, action_scaler = action_scaler, use_gpu=False) #change this if you have one!
# fqe.fit(ope_train_episodes, eval_episodes=ope_test_episodes,
#         tensorboard_dir='runs',
#         n_epochs=50, n_steps_per_epoch=1000, #change if overfitting/underfitting
#         scorers={
#            'init_value': initial_state_value_estimation_scorer,
#             'ave_value': average_value_estimation_scorer,
#            'soft_opc': soft_opc_scorer(return_threshold=0)
#         })

In [15]:
model.save_policy("workflowConfound_CPUonly_redo.pt")
model.save_model("workflowConfoundmodel_CPUonly_redo.pt")

/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:204: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  minimum = torch.tensor(
/home/dasc/anaconda3/envs/jbreeden3.10/lib/python3.10/site-packages/d3rlpy/preprocessing/action_scalers.py:207: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  maximum = torch.tensor(


In [16]:
# from d3rlpy.torch_utility import to_cpu
# to_cpu(model)
# model.save_policy("workflowConfoundCPU.pt")
# model.save_model("workflowConfoundmodelCPU.pt")